Created by: Emmanuel Castillo [ecastillot@unal.edu.co]

Running in Google Collab... 

(Steps 1 and 2 are necessary for google collab executions)

# 1. Installations on the pc

## 1.1. Install obspy

In [1]:
pip install obspy

     |████████████████████████████████| 24.7MB 1.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for obspy: filename=obspy-1.2.2-cp36-cp36m-linux_x86_64.whl size=21665560 sha256=0c3975f8e68c3e19982c730ecade50ba6abce22412df27c3828dac6606ee8745
  Stored in directory: /root/.cache/pip/wheels/94/e0/86/44fd4cea7661f42431c8f6d030f2758ff275cccffcbe8fa2b8
Successfully built obspy


## 1.2 Restart the kernel

**IMPORTANT!!!! : You must restart the kernel**

In [ ]:
import os
os._exit(00)

# 2. Mount a GDrive disk


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3. Import 

## 3.1 Modules


In [2]:
import sys
SANT_module_path = "/content/drive/Shareddrives/SANoiseToolkits/opt/SANoiseToolkits"
sys.path.insert(0,SANT_module_path)


In [3]:
print(sys.path)

['/content/drive/Shareddrives/SANoiseToolkits/opt/SANoiseToolkits', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


## 3.2 Libraries

In [4]:
from obspy.clients.fdsn import Client as FDSN_Client
from obspy.core.utcdatetime import UTCDateTime
from noise_toolkits.mass_downloader.restrictions import (DownloadRestrictions,
                                                          PPSDrestrictions)
from noise_toolkits.mass_downloader.downloader import MassPPSD

# 4 Execute

You need to start a client object

In [5]:
client_tuple = ('fdsn',FDSN_Client('http://sismo.sgc.gov.co:8080'))

Initialize a DownloadRestrictions object

In [6]:
dldR = DownloadRestrictions(network="CM", 
                                station="BAR2",
                                location="00", 
                                channel="*",
                                starttime=UTCDateTime(2019, 1, 1,0),
                                endtime=UTCDateTime(2019, 1, 2,0),
                                chunklength=86400,
                                overlap=None,
                                exclude=[("*","DBB,BAR2","10","HH[ENZ]")],
                                plot_trace=True)

Initialize a PPSDRestrictions object

In [7]:
ppsdR = PPSDrestrictions(skip_on_gaps=False,
                            db_bins=(-200, -50, 1.0), 
                            ppsd_length=3600.0, 
                            overlap=0.5, 
                            special_handling=None, 
                            period_smoothing_width_octaves=1.0, 
                            period_step_octaves=0.125, 
                            period_limits=None)

Define the output directory

In [8]:
my_storage = "/content/drive/Shareddrives/SANoiseToolkits/opt/SANoiseToolkits/examples/downloader/SANL_results"

Initialize MassPPSD session

In [9]:
massppsd = MassPPSD(client_tuple=client_tuple,
                        dld_restrictions = dldR,
                        my_storage=my_storage)

You can create your xml according to the dld_restrictions, after you always will use this.

In [10]:
xml_file = "/content/drive/Shareddrives/SANoiseToolkits/opt/SANoiseToolkits/examples/public_CM.xml"
myxml = massppsd.create_inventory(from_xml=xml_file)

Inventory created at 2020-11-15T16:07:07.651972Z
	Created by: ObsPy 1.2.2
		    https://www.obspy.org
	Sending institution: SeisComP3 (SGC)
	Contains:
		Networks (1):
			CM
		Stations (1):
			CM.BAR2 (Barichara, Santander, Colombia)
		Channels (3):
			CM.BAR2.00.HHZ, CM.BAR2.00.HHN, CM.BAR2.00.HHE


In [11]:
massppsd.download(inv_path=myxml,ppsd_restrictions = ppsdR,
                    n_processor=2,concurrent_feature='thread')

2020/11/27 11:10:15[save_trace][ok]:  CM.BAR2.00.HHN__20190101T000000Z__20190102T000000Z.jpg
2020/11/27 11:10:23[save_trace][ok]:  CM.BAR2.00.HHE__20190101T000000Z__20190102T000000Z.jpg
2020/11/27 11:10:15[save_ppsd][ok]:  CM.BAR2.00.HHN__20190101T000000Z__20190102T000000Z.npz 
2020/11/27 11:10:31[save_trace][ok]:  CM.BAR2.00.HHZ__20190101T000000Z__20190102T000000Z.jpg
2020/11/27 11:10:24[save_ppsd][ok]:  CM.BAR2.00.HHE__20190101T000000Z__20190102T000000Z.npz 
2020/11/27 11:10:32[save_ppsd][ok]:  CM.BAR2.00.HHZ__20190101T000000Z__20190102T000000Z.npz 
